# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# new file
new_file_path = 'data/event_datafile_new.csv'

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))

display(pd.DataFrame(full_data_rows_list).head())

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(new_file_path, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.54102E+12,583,Sehr kosmisch,200,1.54224E+12,26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.54102E+12,583,The Big Gundown,200,1.54224E+12,26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.54102E+12,583,Marry Me,200,1.54224E+12,26
3,,Logged In,Wyatt,M,0,Scott,,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.54087E+12,563,,200,1.54225E+12,9
4,,Logged In,Austin,M,0,Rosales,,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.54106E+12,521,,200,1.54225E+12,12


In [4]:
# check the number of rows in your csv file
with open(new_file_path, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()


#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS studentdb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('studentdb')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Helper Functions

In [8]:
def drop_table(table_name: str) -> None:
    """
    Drops a table given its name
    
    :param table_name: table name
    """
    query = f"DROP TABLE IF EXISTS {table_name}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


In [9]:
def create_table(table_name: str, ordered_cols: dict, partition_keys:str, clustering_cols: str) -> None:
    """
    Creates a table given name, columns, partition keys, and clustering columns
    
    :patam table_name: table name
    :param ordered_cols: all table columns ordered and their respective data types
    :param partition_keys: partition keys
    :param clusering_cols: clustering columns
    """
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
    
    for col_name, data_type in ordered_cols.items():
        query += f"{col_name} {data_type}, "
    
    query += f"PRIMARY KEY ({partition_keys}, {clustering_cols}))"
    
    try:
        session.execute(query)
    except Exception as e:
        print(e)


In [10]:
def _prepare_params(line, ordered_cols, insert_order):
    
    ordered_keys = list(ordered_cols.keys())
    
    params = []
    for idx in range(len(insert_order)):
        if ordered_cols[ordered_keys[idx]] == "int":
            params.append(int(line[insert_order[idx]]))
        elif ordered_cols[ordered_keys[idx]] == "float":
            params.append(float(line[insert_order[idx]]))
        else:
            params.append(line[insert_order[idx]])
    
    return params
    

In [11]:
def ingest_event_data(file_path: str, table_name: str, ordered_cols: dict, insert_order: list) -> None:
    """
    Ingests every raw in the dataset into the table
    
    :param file_path: file path of the processed event logs dataset
    :param table_name: name of table
    :param ordered_cols: all table columns ordered and their respective data types
    """
    
    ordered_keys = list(ordered_cols.keys())
        
    with open(file_path, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"INSERT INTO {table_name} ("
            for idx, col_name in enumerate(ordered_keys):
                if idx < len(ordered_keys)-1:
                    query += f"{col_name}, "
                else:
                    query += f"{col_name}) "
            query += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                                    
            session.execute(
                query, 
                _prepare_params(line, ordered_cols, insert_order)
            )


In [12]:
def search_query(query: str) -> pd.DataFrame:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    return pd.DataFrame(rows)


## Query 1

### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

```
SELECT artist_name, song_title, length FROM session_plays WHERE session_id = 338 AND item_in_session = 4
```

In [13]:

table_name = "session_plays"

# Drop Table if exists
drop_table(table_name)

"""
To query song data by the `session_id` and `item_in_session`, the `session_id` is used as a partition key \
to make sure all session items in a single session exist in a single node.
The `item_in_session` is then used to sort data and make sure every combination is unique.
"""

ordered_cols = {
    "session_id": "int",
    "item_in_session": "int",
    "artist_name": "text",
    "song_title": "text",
    "length": "float",
    "user_id": "int",
    "first_name": "text",
    "last_name": "text",
    "gender": "text",
    "level": "text",
    "location": "text"
}

# Create Table
create_table(
    table_name,
    ordered_cols,
    partition_keys="(session_id)",
    clustering_cols="item_in_session"
)

# Ingest Event Data
ingest_event_data(new_file_path, table_name, ordered_cols, insert_order=[8, 3, 0, 9, 5, 10, 1, 4, 2, 6, 7])


In [14]:
search_query(
    "SELECT artist_name, song_title, length FROM session_plays WHERE session_id = 338 AND item_in_session = 4"
)


,artist_name,song_title,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```
SELECT artist_name, song_title, first_name, last_name FROM user_sessions WHERE user_id = 10 AND session_id = 182
```

In [15]:

table_name = "user_sessions"

# Drop Table if exists
drop_table(table_name)

"""
To query song and user data by the `user_id and `session_id`, they are used as partition keys to make sure every user session lie in a single node.
The `item_in_session` is then used to sort data and make sure every combination is unique.
"""

ordered_cols = {
    "user_id": "int",
    "session_id": "int",
    "item_in_session": "int",
    "artist_name": "text",
    "song_title": "text",
    "first_name": "text",
    "last_name": "text",
    "gender": "text",
    "length": "float",
    "level": "text",
    "location": "text"
}

# Create Table
create_table(
    table_name,
    ordered_cols,
    partition_keys="(user_id, session_id)",
    clustering_cols="item_in_session"
)

# Ingest Event Data
ingest_event_data(new_file_path, table_name, ordered_cols, insert_order=[10, 8, 3, 0, 9, 1, 4, 2, 5, 6, 7])
                    

In [16]:
search_query(
    "SELECT artist_name, song_title, first_name, last_name FROM user_sessions WHERE user_id = 10 AND session_id = 182"
)


,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```
SELECT first_name, last_name FROM song_fans WHERE song_title = 'All Hands Against His Own'
```

In [17]:

table_name = "song_fans"

# Drop Table if exists
drop_table(table_name)

"""
To find all fans of specific songs, the `song_title` is to partition the data among nodes to be get all data from a single node.
As we expect to have multiple users hearing the same song, we use user_id to compose the primary key with the song.
"""

ordered_cols = {
    "song_title": "text",
    "user_id": "int",
    "first_name": "text",
    "last_name": "text",
    "session_id": "int",
    "item_in_session": "int",
    "artist_name": "text",
    "gender": "text",
    "length": "float",
    "level": "text",
    "location": "text"
}

# Create Table
create_table(
    table_name,
    ordered_cols,
    partition_keys="(song_title)",
    clustering_cols="user_id"
)

# Ingest Event Data
ingest_event_data(new_file_path, table_name, ordered_cols, insert_order=[9, 10, 1, 4, 8, 3, 0, 2, 5, 6, 7])


In [18]:
search_query(
    "SELECT first_name, last_name FROM song_fans WHERE song_title = 'All Hands Against His Own'"
)


,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
drop_table("session_plays")
drop_table("user_sessions")
drop_table("song_fans")


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()